In [4]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry

ENDPOINT = "https://southcentralus.api.cognitive.microsoft.com"

# Replace with a valid key
training_key = "0ce3b236d6894639905fb6078708f236"
prediction_key = "57c2350f261b4cf0aec943bcf893e2d8"

trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)

# Create a new project
#print ("Creating project...")
#project = trainer.create_project("RateRoomsBalancedSamples")

project_id = '40412ce6-fe45-4fc2-b949-9bb6fe28abf2'

In [2]:
# Make all tags in the new project. ONLY DO THIS ONCE
trainer.create_tag(project.id, "very low")
trainer.create_tag(project.id, "low")
trainer.create_tag(project.id, "average")
trainer.create_tag(project.id, "high")
trainer.create_tag(project.id, "very high")

In [5]:
trainer.get_tags(project_id)

In [6]:
# tag ids gotten from https://southcentralus.dev.cognitive.microsoft.com/docs/services/Custom_Vision_Training_2.2/operations/5b63704940d86a0fb87aab50/console
very_low_tag = "3174baf0-60d1-45fc-abd9-a206bc0c7bfd"
low_tag = "acae4915-0bec-408b-9248-079afb6b13c3"
average_tag = "9ac3099b-055a-4162-9a70-0eeeb0711050"
high_tag = "80423186-8716-4d05-93f2-09d4b666bd1e"
very_high_tag = "7ebd69b5-03fd-45b9-aa00-dea002e41777"

In [9]:
#
# load all data into python 
#

import pandas as pd

# load img urls and labels into two separate lists
dirty_data = open('pretty.json','r')

fileNames = []
labels = []

# limit the number of high and avg samples so
# there wont be too many of them
avg_count = 0
high_count = 0
MAX_COUNT = 1400


for line in dirty_data:
    
    # every lin in file is a json containing a single sample
    dataSample = pd.read_json(line)
    try:
        url = dataSample.content[0]
        label = dataSample.annotation.labels[0]
        
    # skip sample if label is missing
    except IndexError:
        continue
        
    if(label == 'average'):
        if(avg_count > MAX_COUNT):
            continue
        avg_count +=1
        
    elif(label =='high'):
        if(high_count > MAX_COUNT):
            continue
        high_count +=1
    
    fileNames.append(url)
    labels.append(label)
    
# convert to dataframe and make sure the labeling is correct
dataRaw = {'url':fileNames, 'label': labels}
data = pd.DataFrame(dataRaw)
print(data.label.unique())

['average' 'high' 'low' 'very low' 'very high']


In [10]:
# verify shape and integrity of data
print(data.shape)
data.isna().sum()
data['label'].value_counts()

(4306, 2)


low          1410
high         1401
average      1401
very high      59
very low       35
Name: label, dtype: int64

In [11]:
label_ids = []

# convert label words into label ids as needed by customVision API
for label in labels:
    if label == 'very low':
        label_ids.append(very_low_tag)
    elif label == 'low':
        label_ids.append(low_tag)
    elif label == 'average':
        label_ids.append(average_tag)
    elif label == 'high':
        label_ids.append(high_tag)
    elif label == 'very high':
        label_ids.append(very_high_tag)
    else:
        print(label)
        
# verify integrity
print(len(label_ids))

4306


In [13]:
#base_image_url = "https://adacloud.blob.core.windows.net/rails/uploads/photo/file/"

num_files = len(fileNames)
index = 0


print ("Adding images...")

# batch of samples to upload to project. Maximum size 64
# uploads once lists size is 64 and then 
images = []

# run through every file
while index < num_files:
    
    #batch has reached a max size of 64
    if(len(images) % 64 == 0 and len(images) != 0):
        trainer.create_images_from_urls(project_id, images=images) # upload batch
        print('Batch ' + str(index) + 'uploaded')
        images = [] # empty batch again
        

    # create new batch sample
    image = ImageUrlCreateEntry(url= fileNames[index], tag_ids=[label_ids[index]] )
    images.append(image) # add sample to batch
    index += 1

# upload leftover batch that is not full yet
if(len(images) != 0):
    trainer.create_images_from_urls(project_id, images=images)

Adding images...
Batch 64uploaded
Batch 128uploaded
Batch 192uploaded
Batch 256uploaded
Batch 320uploaded
Batch 384uploaded
Batch 448uploaded
Batch 512uploaded
Batch 576uploaded
Batch 640uploaded
Batch 704uploaded
Batch 768uploaded
Batch 832uploaded
Batch 896uploaded
Batch 960uploaded
Batch 1024uploaded
Batch 1088uploaded
Batch 1152uploaded
Batch 1216uploaded
Batch 1280uploaded
Batch 1344uploaded
Batch 1408uploaded
Batch 1472uploaded
Batch 1536uploaded
Batch 1600uploaded
Batch 1664uploaded
Batch 1728uploaded
Batch 1792uploaded
Batch 1856uploaded
Batch 1920uploaded
Batch 1984uploaded
Batch 2048uploaded
Batch 2112uploaded
Batch 2176uploaded
Batch 2240uploaded
Batch 2304uploaded
Batch 2368uploaded
Batch 2432uploaded
Batch 2496uploaded
Batch 2560uploaded
Batch 2624uploaded
Batch 2688uploaded
Batch 2752uploaded
Batch 2816uploaded
Batch 2880uploaded
Batch 2944uploaded
Batch 3008uploaded
Batch 3072uploaded
Batch 3136uploaded
Batch 3200uploaded
Batch 3264uploaded
Batch 3328uploaded
Batch 3392u

In [24]:
# This has no use
# Attempted to upload local files but did not use
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry

images = []

image = ImageFileCreateEntry(name='test.jpg', tag_ids=['3174baf0-60d1-45fc-abd9-a206bc0c7bfd'])
images.append(image)
trainer.create_images_from_files(project_id, images=images)
